## 이벤트 플레그 변환 규칙

1.  TCN 모델과 yolo 객채탐지 모델을 섞어 판단
    1.  우선 TCN 예측 과 yolo box를 합친 df를 생성
2.  각 행동 (A/S/D)의 시작과 끝 규칙을 정함
    1.  A 행동
        *   정의: 상자를 꺼낸 후 연다.
        *   시작: box count가 0에서 증가
        *   끝: box count가 4 그리거 open 또한 4
    2.  S 행동
        *   정의: 상자에 물건을 넣는다
        *   시작: box의 empty가 4에서 감소
        *   끝: empty가 0 그리고 full이 4
    3.  D 행동
        *   정의: 상자를 닫고 트레이에 넣는다
        *   시작: open 4감소 및 box count 감소
        *   끝: box count 0이 됨

3.  우선 TCN에서 추출된 구간(러프하게)을 추출하여 그 구간 속에서 box의 객채 수가 각 조건에 맞는 위치를 찾아 플레그로 설정
4.  실제 상황은 한번에 4개를 같이 작업, 손에 가려 탐지가 안되는 경우, 다른 물체에 가려 추적이 안되는 경우, 오탐으로 인하여 잘못 카운트 되는 경우 등  여러가지 오탐 케이스들이 많아 위의 조건을 통해서도 전부 탐지가 되지는 않는다.

In [16]:
import pandas as pd
import numpy as np
import os

def build_events_from_tcn_yolo(
    tcn_path: str,
    yolo_path: str,
    out_csv_path: str | None = None,
    fps: float = 30.0,
    min_seg_len: int = 5,
):
    """
    TCN 예측 CSV + YOLO 상태 CSV를 이용해서
    A/S/D 이벤트 플래그 CSV를 생성.

    입력:
        tcn_path  : video_xxx_pred.csv
            - 컬럼: A, S, D (0/1 또는 확률)
            - 행 인덱스 = frame_idx
        yolo_path : video_xxx_yolo_states.csv
            - 컬럼: frame_idx, box_count, open_count, closed_count, full_count, empty_count, ...

    출력:
        events_df: frame_idx, time_sec, flag_id, flag_key
        (out_csv_path가 주어지면 파일로 저장도 함)
    """

    # 1) CSV 로드 & 길이 맞추기
    df_tcn = pd.read_csv(tcn_path)
    df_yolo = pd.read_csv(yolo_path)

    n = min(len(df_tcn), len(df_yolo))
    df_tcn = df_tcn.iloc[:n].reset_index(drop=True)
    df_yolo = df_yolo.iloc[:n].reset_index(drop=True)

    # 한 DataFrame으로 합치고 frame_idx 추가
    df = pd.concat([df_tcn, df_yolo], axis=1)
    df["frame_idx"] = np.arange(n)

    # ------------------------------------------------
    # 2) TCN 기반 1차 라벨 (tcn_label)
    # ------------------------------------------------
    def row_to_tcn_label(row):
        vals = [row["A"], row["S"], row["D"]]
        labels = ["A", "S", "D"]
        maxv = max(vals)
        if maxv <= 0:
            return "idle"
        # 동률이면 A > S > D 순으로
        for lab, v in zip(labels, vals):
            if v == maxv:
                return lab

    df["tcn_label"] = df.apply(row_to_tcn_label, axis=1)

    # ------------------------------------------------
    # 3) YOLO 기반 패턴 힌트 (A_like, S_like, D_like)
    # ------------------------------------------------
    # 없으면 0으로 채우기
    bc = df.get("box_count", pd.Series([0] * n))
    oc = df.get("open_count", pd.Series([0] * n))
    cc = df.get("closed_count", pd.Series([0] * n))
    fc = df.get("full_count", pd.Series([0] * n))
    ec = df.get("empty_count", pd.Series([0] * n))

    diff_bc = bc.diff().fillna(0)
    diff_oc = oc.diff().fillna(0)
    diff_fc = fc.diff().fillna(0)
    diff_ec = ec.diff().fillna(0)

    # A 단계 패턴
    yolo_A_like = (
        ((bc >= 1) & (oc >= 1) & (fc == 0))  # 비어 있고 열려있는 상자
        | (diff_bc > 0)                      # 상자 개수 증가
        | (diff_oc > 0)                      # 열린 상자 증가
    )

    # S 단계 패턴
    yolo_S_like = (
        ((oc >= 1) & (fc >= 1))              # 열려 있고 내용물도 있음
        | ((diff_fc > 0) & (diff_ec <= 0))   # full 증가 & empty 유지/감소
    )

    # D 단계 패턴
    yolo_D_like = (
        (cc >= 1)                            # 닫힌 상자 보임
        | (diff_bc < 0)                      # 상자 개수 감소
        | (diff_oc < 0)                      # 열린 상자 감소
    )

    df["yolo_A_like"] = yolo_A_like
    df["yolo_S_like"] = yolo_S_like
    df["yolo_D_like"] = yolo_D_like

    # ------------------------------------------------
    # 4) TCN + YOLO 점수 융합 → fused_label_raw
    # ------------------------------------------------
    w_tcn = 2.0
    w_yolo = 1.0

    scores_A = w_tcn * (df["tcn_label"] == "A") + w_yolo * (yolo_A_like.astype(float))
    scores_S = w_tcn * (df["tcn_label"] == "S") + w_yolo * (yolo_S_like.astype(float))
    scores_D = w_tcn * (df["tcn_label"] == "D") + w_yolo * (yolo_D_like.astype(float))
    scores_idle = w_tcn * (df["tcn_label"] == "idle")

    fused_raw = []
    for a, s, d, i in zip(scores_A, scores_S, scores_D, scores_idle):
        arr = [a, s, d, i]
        idx = int(np.argmax(arr))
        fused_raw.append(["A", "S", "D", "idle"][idx])

    df["fused_label_raw"] = fused_raw

    # ------------------------------------------------
    # 5) 너무 짧은 A/S/D 세그먼트를 idle로 제거 → fused_label
    # ------------------------------------------------
    labels = df["fused_label_raw"].tolist()

    # 라벨이 연속된 구간(세그먼트) 찾기
    segs = []
    cur_label = labels[0]
    start = 0
    for i in range(1, len(labels)):
        if labels[i] != cur_label:
            segs.append((cur_label, start, i - 1))
            cur_label = labels[i]
            start = i
    segs.append((cur_label, start, len(labels) - 1))

    labels_smooth = labels.copy()
    for lab, s, e in segs:
        length = e - s + 1
        if lab != "idle" and length < min_seg_len:
            # 너무 짧은 구간은 idle로 덮어버림
            for i in range(s, e + 1):
                labels_smooth[i] = "idle"

    df["fused_label"] = labels_smooth

    # ------------------------------------------------
    # 6) fused_label → 이벤트 플래그 (A/S/D) 로 변환
    #    각 행동별로 "가장 긴 세그먼트"만 사용
    # ------------------------------------------------
    flag_id_map = {"A": 1, "S": 2, "D": 3}
    events = []

    for lab in ["A", "S", "D"]:
        # lab에 해당하는 세그먼트들 찾기
        segs_lab = []
        start = None
        for i, l in enumerate(labels_smooth):
            if l == lab and start is None:
                start = i
            elif l != lab and start is not None:
                segs_lab.append((start, i - 1))
                start = None
        if start is not None:
            segs_lab.append((start, len(labels_smooth) - 1))

        if not segs_lab:
            # 이 행동(A/S/D)이 전혀 없으면 skip
            continue

        # 가장 긴 구간만 사용 (메인 단계)
        best_start, best_end = max(segs_lab, key=lambda x: x[1] - x[0])

        events.append(
            {
                "frame_idx": best_start,
                "time_sec": best_start / fps,
                "flag_id": flag_id_map[lab],
                "flag_key": lab,
            }
        )
        events.append(
            {
                "frame_idx": best_end,
                "time_sec": best_end / fps,
                "flag_id": flag_id_map[lab],
                "flag_key": lab,
            }
        )

    events_df = pd.DataFrame(events).sort_values(["flag_id", "frame_idx"]).reset_index(drop=True)

    # ------------------------------------------------
    # 7) 저장 옵션
    # ------------------------------------------------
    if out_csv_path is not None:
        base = os.path.basename(tcn_path)  
        filename = base.replace("_pred", "") 
        filename = filename.replace(".csv", "_flag.csv")
        save_path = os.path.join(out_csv_path, filename)
        events_df.to_csv(save_path, index=False, encoding="utf-8-sig")
        print(f"[INFO] Saved events CSV to: {save_path}")

    return df, events_df


In [4]:
# 예: video_normal_new_001 세트
tcn_path  = r"test_video\out_TCN\video_normal_new_001_pred.csv"
yolo_path = r"test_video\out_yolo\video_normal_new_001(normal)_yolo_states.csv"

merged_df, events_df = build_events_from_tcn_yolo(
    tcn_path,
    yolo_path,
    out_csv_path=r"test_video\out_pred",
    fps=7,       # 필요하면 조정
    min_seg_len=5,  # 노이즈 제거 최소 길이
)

print(events_df)

merged_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\yolo_to_tcn_video_normal_new_001.csv")
events_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\video_normal_new_001_flage.csv")


[INFO] Saved events CSV to: test_video\out_pred\video_normal_new_001_flag.csv
   frame_idx   time_sec  flag_id flag_key
0          0   0.000000        1        A
1         39   5.571429        1        A
2        140  20.000000        2        S
3        234  33.428571        2        S
4        110  15.714286        3        D
5        139  19.857143        3        D


In [5]:
# 예: video_normal_new_001 세트
tcn_path  = r"test_video\out_TCN\video_normal_new_002_pred.csv"
yolo_path = r"test_video\out_yolo\video_normal_new_002(fast)_yolo_states.csv"

merged_df, events_df = build_events_from_tcn_yolo(
    tcn_path,
    yolo_path,
    out_csv_path=r"test_video\out_pred",
    fps=7,       # 필요하면 조정
    min_seg_len=5,  # 노이즈 제거 최소 길이
)

print(events_df)
merged_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\yolo_to_tcn_video_normal_new_002.csv")
events_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\video_normal_new_002_flage.csv")


[INFO] Saved events CSV to: test_video\out_pred\video_normal_new_002_flag.csv
   frame_idx   time_sec  flag_id flag_key
0         45   6.428571        1        A
1         74  10.571429        1        A
2         85  12.142857        2        S
3        169  24.142857        2        S
4        170  24.285714        3        D
5        234  33.428571        3        D


In [6]:
# 예: video_normal_new_001 세트
tcn_path  = r"test_video\out_TCN\video_normal_new_003_pred.csv"
yolo_path = r"test_video\out_yolo\video_normal_new_003_yolo_states.csv"

merged_df, events_df = build_events_from_tcn_yolo(
    tcn_path,
    yolo_path,
    out_csv_path=r"test_video\out_pred",
    fps=7,       # 필요하면 조정
    min_seg_len=5,  # 노이즈 제거 최소 길이
)

print(events_df)

merged_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\yolo_to_tcn_video_normal_new_003.csv")
events_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\video_normal_new_003_flage.csv")

[INFO] Saved events CSV to: test_video\out_pred\video_normal_new_003_flag.csv
   frame_idx   time_sec  flag_id flag_key
0        120  17.142857        1        A
1        164  23.428571        1        A
2        175  25.000000        2        S
3        314  44.857143        2        S
4        335  47.857143        3        D
5        434  62.000000        3        D


In [7]:
# 예: video_normal_new_001 세트
tcn_path  = r"test_video\out_TCN\video_missing1_new_001_pred.csv"
yolo_path = r"test_video\out_yolo\video_missing1_new_001_yolo_states.csv"

merged_df, events_df = build_events_from_tcn_yolo(
    tcn_path,
    yolo_path,
    out_csv_path=r"test_video\out_pred",
    fps=7,       # 필요하면 조정
    min_seg_len=5,  # 노이즈 제거 최소 길이
)

print(events_df)

merged_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\yolo_to_tcn_video_missing1_new_001.csv")
events_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\video_missing1_new_001_flage.csv")

[INFO] Saved events CSV to: test_video\out_pred\video_missing1_new_001_flag.csv
   frame_idx   time_sec  flag_id flag_key
0         40   5.714286        2        S
1        119  17.000000        2        S
2        125  17.857143        3        D
3        184  26.285714        3        D


In [8]:
tcn_path  = r"test_video\out_TCN\video_missing1_new_002_pred.csv"
yolo_path = r"test_video\out_yolo\video_missing1_new_002_yolo_states.csv"

merged_df, events_df = build_events_from_tcn_yolo(
    tcn_path,
    yolo_path,
    out_csv_path=r"test_video\out_pred",
    fps=7,       # 필요하면 조정
    min_seg_len=5,  # 노이즈 제거 최소 길이
)

print(events_df)

merged_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\yolo_to_tcn_video_missing1_new_002.csv")
events_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\video_missing1_new_002_flage.csv")

[INFO] Saved events CSV to: test_video\out_pred\video_missing1_new_002_flag.csv
   frame_idx   time_sec  flag_id flag_key
0         70  10.000000        1        A
1         94  13.428571        1        A
2          0   0.000000        2        S
3         19   2.714286        2        S
4        165  23.571429        3        D
5        199  28.428571        3        D


In [9]:
tcn_path  = r"test_video\out_TCN\video_missing1_new_003_pred.csv"
yolo_path = r"test_video\out_yolo\video_missing1_new_003_yolo_states.csv"

merged_df, events_df = build_events_from_tcn_yolo(
    tcn_path,
    yolo_path,
    out_csv_path=r"test_video\out_pred",
    fps=7,       # 필요하면 조정
    min_seg_len=5,  # 노이즈 제거 최소 길이
)

print(events_df)

merged_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\yolo_to_tcn_video_missing1_new_003.csv")
events_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\video_missing1_new_003_flage.csv")

[INFO] Saved events CSV to: test_video\out_pred\video_missing1_new_003_flag.csv
   frame_idx   time_sec  flag_id flag_key
0         40   5.714286        1        A
1         89  12.714286        1        A
2        110  15.714286        2        S
3        194  27.714286        2        S
4          0   0.000000        3        D
5         39   5.571429        3        D


In [10]:
tcn_path  = r"test_video\out_TCN\video_missing2_new_001_pred.csv"
yolo_path = r"test_video\out_yolo\video_missing2_new_001_yolo_states.csv"

merged_df, events_df = build_events_from_tcn_yolo(
    tcn_path,
    yolo_path,
    out_csv_path=r"test_video\out_pred",
    fps=7,       # 필요하면 조정
    min_seg_len=5,  # 노이즈 제거 최소 길이
)

print(events_df)

merged_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\yolo_to_tcn_video_missing2_new_001.csv")
events_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\video_missing2_new_001_flage.csv")

[INFO] Saved events CSV to: test_video\out_pred\video_missing2_new_001_flag.csv
   frame_idx   time_sec  flag_id flag_key
0        110  15.714286        1        A
1        134  19.142857        1        A
2         60   8.571429        3        D
3         74  10.571429        3        D


In [11]:
tcn_path  = r"test_video\out_TCN\video_missing2_new_002_pred.csv"
yolo_path = r"test_video\out_yolo\video_missing2_new_002_yolo_states.csv"

merged_df, events_df = build_events_from_tcn_yolo(
    tcn_path,
    yolo_path,
    out_csv_path=r"test_video\out_pred",
    fps=7,       # 필요하면 조정
    min_seg_len=5,  # 노이즈 제거 최소 길이
)

print(events_df)

merged_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\yolo_to_tcn_video_missing2_new_002.csv")
events_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\video_missing2_new_002_flage.csv")

[INFO] Saved events CSV to: test_video\out_pred\video_missing2_new_002_flag.csv
   frame_idx   time_sec  flag_id flag_key
0        110  15.714286        2        S
1        149  21.285714        2        S
2        100  14.285714        3        D
3        104  14.857143        3        D


In [12]:
tcn_path  = r"test_video\out_TCN\video_missing2_new_003_pred.csv"
yolo_path = r"test_video\out_yolo\video_missing2_new_003_yolo_states.csv"

merged_df, events_df = build_events_from_tcn_yolo(
    tcn_path,
    yolo_path,
    out_csv_path=r"test_video\out_pred",
    fps=7,       # 필요하면 조정
    min_seg_len=5,  # 노이즈 제거 최소 길이
)

print(events_df)

merged_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\yolo_to_tcn_video_missing2_new_003.csv")
events_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\video_missing2_new_003_flage.csv")

[INFO] Saved events CSV to: test_video\out_pred\video_missing2_new_003_flag.csv
   frame_idx   time_sec  flag_id flag_key
0         75  10.714286        3        D
1        164  23.428571        3        D


In [15]:
# 예: video_normal_new_001 세트
tcn_path  = r"test_video\out_TCN\video_ idle_001_pred.csv"
yolo_path = r"test_video\out_yolo\video_ idle_001_yolo_states.csv"

merged_df, events_df = build_events_from_tcn_yolo(
    tcn_path,
    yolo_path,
    out_csv_path=r"test_video\out_pred",
    fps=7.5,       # 필요하면 조정
    min_seg_len=5,  # 노이즈 제거 최소 길이
)

print(events_df)

merged_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\yolo_to_tcn_video_idle_new_001.csv")
events_df.to_csv(r"G:\GitProjects\sessac_project\test_data\test_pred\video_idle_new_001_flage.csv")

[INFO] Saved events CSV to: test_video\out_pred\video_ idle_001_flag.csv
   frame_idx   time_sec  flag_id flag_key
0        200  26.666667        3        D
1        239  31.866667        3        D


## 최종 이벤트 플레그 평가

In [2]:
import os
import pandas as pd

# ----------------------------------------------------------
# 1. 설정
# ----------------------------------------------------------
GT_DIR   = r"G:\GitProjects\sessac_project\test_data\test_flagle"
PRED_DIR = r"G:\GitProjects\sessac_project\test_data\test_pred"

EVENT_KEYS = ["A", "S", "D"]


# ----------------------------------------------------------
# 2. 유틸: df → interval list(A, S, D만)
# ----------------------------------------------------------
def intervals_from_events_df(df):
    """
    flag_key, frame_idx, flag_id 컬럼이 있다고 가정.
    A/S/D만 골라서 flag_id별로 최소~최대 frame_idx를 구간으로 사용.
    """
    df2 = df[df["flag_key"].isin(EVENT_KEYS)].copy()
    df2 = df2.sort_values(["flag_id", "frame_idx"])
    
    intervals = []
    for (fid, key), grp in df2.groupby(["flag_id", "flag_key"]):
        start = int(grp["frame_idx"].min())
        end   = int(grp["frame_idx"].max())
        if start == end:
            end = start + 1  # 최소 1 프레임폭
        intervals.append({"key": key, "start": start, "end": end})
    return intervals


# ----------------------------------------------------------
# 3. IoU 계산
# ----------------------------------------------------------
def iou_interval(a_start, a_end, b_start, b_end):
    inter = max(0, min(a_end, b_end) - max(a_start, b_start))
    union = max(a_end, b_end) - min(a_start, b_start)
    return inter / union if union > 0 else 0.0


# ----------------------------------------------------------
# 4. GT 기준으로만 파일 매칭 (test_pred의 다른 csv 무시)
# ----------------------------------------------------------
def scan_csv_pairs(gt_dir, pred_dir):
    """
    gt_dir 에 있는
      video_{이벤트이름}_new_{번호}_events.csv
    를 기준으로,
    pred_dir 에 동일 베이스 이름의
      video_{이벤트이름}_new_{번호}_flage.csv
    가 있을 때만 pair로 묶는다.
    """
    gt_files = [
        f for f in os.listdir(gt_dir)
        if f.startswith("video_") and f.endswith("_events.csv")
    ]

    pairs = []

    for gt_name in gt_files:
        key = gt_name.replace("_events.csv", "")  # video_xxx_new_yyy
        pred_name = key + "_flage.csv"           # video_xxx_new_yyy_flage.csv

        gt_path   = os.path.join(gt_dir, gt_name)
        pred_path = os.path.join(pred_dir, pred_name)

        if os.path.exists(pred_path):
            pairs.append((key, gt_path, pred_path))
        else:
            print(f"[WARN] Pred not found for {gt_name} -> 기대한 파일: {pred_name}")

    return pairs


# ----------------------------------------------------------
# 5. 이벤트 단위 metrics (IoU ≥ threshold)
# ----------------------------------------------------------
def event_level_metrics(pairs, iou_thr=0.5):
    """
    pairs: (key, gt_path, pred_path) 리스트
    key 하나당 A/S/D 각 1개씩 이벤트가 있다고 가정.
    """
    results = {}
    total = {k: {"tp": 0, "fp": 0, "fn": 0} for k in EVENT_KEYS}

    for key, gt_path, pred_path in pairs:
        gt_df   = pd.read_csv(gt_path)
        pred_df = pd.read_csv(pred_path)

        gt_ints   = intervals_from_events_df(gt_df)
        pred_ints = intervals_from_events_df(pred_df)

        per_video = {}

        for k in EVENT_KEYS:
            g_list = [e for e in gt_ints   if e["key"] == k]
            p_list = [e for e in pred_ints if e["key"] == k]

            if not g_list or not p_list:
                # 한쪽에만 있는 경우 예외 처리
                if g_list and not p_list:
                    total[k]["fn"] += 1
                    per_video[k] = {
                        "gt_interval": (g_list[0]["start"], g_list[0]["end"]),
                        "pred_interval": None,
                        "iou": 0.0,
                        "match": False
                    }
                elif p_list and not g_list:
                    total[k]["fp"] += 1
                    per_video[k] = {
                        "gt_interval": None,
                        "pred_interval": (p_list[0]["start"], p_list[0]["end"]),
                        "iou": 0.0,
                        "match": False
                    }
                else:
                    per_video[k] = {
                        "gt_interval": None,
                        "pred_interval": None,
                        "iou": 0.0,
                        "match": False
                    }
                continue

            g = g_list[0]
            p = p_list[0]

            iou = iou_interval(g["start"], g["end"], p["start"], p["end"])
            match = (iou >= iou_thr)

            if match:
                total[k]["tp"] += 1
            else:
                total[k]["fp"] += 1
                total[k]["fn"] += 1

            per_video[k] = {
                "gt_interval": (g["start"], g["end"]),
                "pred_interval": (p["start"], p["end"]),
                "iou": round(iou, 4),
                "match": match
            }

        results[key] = per_video

    # 클래스별 F1
    per_class = {}
    for k in EVENT_KEYS:
        tp = total[k]["tp"]
        fp = total[k]["fp"]
        fn = total[k]["fn"]
        prec = tp / (tp + fp) if tp + fp > 0 else 0.0
        rec  = tp / (tp + fn) if tp + fn > 0 else 0.0
        f1   = 2 * prec * rec / (prec + rec) if prec + rec > 0 else 0.0
        per_class[k] = {
            "tp": tp, "fp": fp, "fn": fn,
            "precision": round(prec,4),
            "recall": round(rec,4),
            "f1": round(f1,4)
        }

    # micro F1
    TP = sum(total[k]["tp"] for k in EVENT_KEYS)
    FP = sum(total[k]["fp"] for k in EVENT_KEYS)
    FN = sum(total[k]["fn"] for k in EVENT_KEYS)

    micro_prec = TP / (TP + FP) if TP + FP > 0 else 0.0
    micro_rec  = TP / (TP + FN) if TP + FN > 0 else 0.0
    micro_f1   = 2 * micro_prec * micro_rec / (micro_prec + micro_rec) if micro_prec + micro_rec > 0 else 0.0

    summary = {
        "precision": round(micro_prec,4),
        "recall": round(micro_rec,4),
        "f1": round(micro_f1,4),
        "TP": TP, "FP": FP, "FN": FN
    }

    return results, per_class, summary


# ----------------------------------------------------------
# 6. 실행
# ----------------------------------------------------------
pairs = scan_csv_pairs(GT_DIR, PRED_DIR)

print("=== 매칭된 파일 쌍 ===")
for key, gt, pred in pairs:
    print(f"{key}:")
    print(f"  GT   : {gt}")
    print(f"  Pred : {pred}")

results, per_class, summary = event_level_metrics(pairs, iou_thr=0.5)

print("\n=== 비디오별 IoU / 매칭 여부 ===")
for key, info in results.items():
    print(f"\n[{key}]")
    for k, d in info.items():
        print(f"  {k}: gt={d['gt_interval']}, pred={d['pred_interval']}, IoU={d['iou']}, match={d['match']}")

print("\n=== 클래스별 F1 ===")
for k, d in per_class.items():
    print(f"{k}: {d}")

print("\n=== 전체 (micro) ===")
print(summary)


[WARN] Pred not found for video_ idle_001_events.csv -> 기대한 파일: video_ idle_001_flage.csv
=== 매칭된 파일 쌍 ===
video_missing1_new_001:
  GT   : G:\GitProjects\sessac_project\test_data\test_flagle\video_missing1_new_001_events.csv
  Pred : G:\GitProjects\sessac_project\test_data\test_pred\video_missing1_new_001_flage.csv
video_missing1_new_002:
  GT   : G:\GitProjects\sessac_project\test_data\test_flagle\video_missing1_new_002_events.csv
  Pred : G:\GitProjects\sessac_project\test_data\test_pred\video_missing1_new_002_flage.csv
video_missing1_new_003:
  GT   : G:\GitProjects\sessac_project\test_data\test_flagle\video_missing1_new_003_events.csv
  Pred : G:\GitProjects\sessac_project\test_data\test_pred\video_missing1_new_003_flage.csv
video_missing2_new_001:
  GT   : G:\GitProjects\sessac_project\test_data\test_flagle\video_missing2_new_001_events.csv
  Pred : G:\GitProjects\sessac_project\test_data\test_pred\video_missing2_new_001_flage.csv
video_missing2_new_002:
  GT   : G:\GitProjects\s